In [8]:
!pip install mxnet-cu101mkl

     |████████████████████████████████| 587.7MB 26kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
from mxnet import gluon, init
from mxnet.gluon import nn

In [0]:
def load_data_fashion_mnist(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load into memory."""
    dataset = gluon.data.vision
    trans = [dataset.transforms.Resize(resize)] if resize else []
    trans.append(dataset.transforms.ToTensor())
    trans = dataset.transforms.Compose(trans )
    mnist_train = dataset.FashionMNIST(train=True).transform_first(trans)
    mnist_test = dataset.FashionMNIST(train=False).transform_first(trans)
    return (gluon.data.DataLoader(mnist_train, batch_size, shuffle=True,
                                  num_workers=4, ),
            gluon.data.DataLoader(mnist_test, batch_size, shuffle=False,
                                  num_workers=4))

In [11]:
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [0]:
net = nn.Sequential()
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))


In [0]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()


In [0]:
def accuracy(y_hat, y):
    if y_hat.shape[1] > 1:
        return ((y_hat.argmax(axis=1) == y.astype('float32')).sum()).astype('float32')
    else:
        return ((y_hat.astype('int32') == y.astype('int32')).sum()).astype('float32')

In [0]:
def eval_accuracy(net, data_iter):

  metric = Accumulator(2)

  for X,y in data_iter:
     metric.add(accuracy(net(X), y), y.size)

  return metric[0] / metric[1]

In [0]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})


In [0]:
def trainch3( net, train_iter,test_iter, loss, epochs, updater):

  animator_trainloss = Animator([],[], "Train Loss")
  animator_trainaccuracy = Animator([], [], "Train Accuracy")
  animator_test_accuracy = Animator([],[],"Test Accuracy")

  for epoch in range(epochs):
    train_metrics = train_epoch(net, train_iter, loss, updater)
    test_acc = eval_accuracy(net, test_iter)
    #animator_trainloss.add_plot(epoch+1, train_metrics[0] )
    #animator_trainaccuracy.add_plot(epoch+1, train_metrics[1])
    animator_test_accuracy.add_plot(epoch+1, float(test_acc.asnumpy()))

In [0]:
class Accumulator:

  def __init__(self, n):

    self.data = [0.0] * n

  def add(self, *args):

    
    self.data = [a+b for a, b in zip(self.data, args)]

  def reset(self):

    self.data = [0] * len(self.data)

  def __getitem__(self,i):

    return self.data[i]

In [0]:
import plotly.graph_objects as go
from IPython import display


class Animator():

  def __init__(self, x, y, z):

    self.fig = go.Figure()
    self.x = x
    self.y = y
    self.z = z

  def add_plot(self, x, y):

    self.x.append(x)

    

    self.y.append(y)

    





    self.fig.add_trace(go.Scatter(x=self.x, y= self.y,
                        mode='lines',
                        ))

    # self.fig.add_trace(go.Scatter(x=y.asnumpy(), y = y.grad.asnumpy(),
    #                     mode='lines+markers',
    #                     name='cosx'))


    self.fig.update_layout(title= self.z,
                            xaxis_title='Epoch',
                           yaxis_title='Metric'+self.z
                           
                           )

    display.clear_output(wait=True)
    display.display(self.fig)

In [0]:
def train_epoch(net, train_iter, loss, updater):

  metric = Accumulator(3)
  if isinstance(updater, gluon.Trainer):
      updater = updater.step


  for X,y in train_iter:

    with mx.autograd.record():
          y_hat = net(X)
          l = loss(y_hat , y)

    l.backward()

    updater(X.shape[0])

    metric.add((l.sum().astype(float)), accuracy(y_hat, y).astype(float), y.size)

  return float((metric[0]/metric[2]).asnumpy()), float((metric[1]/metric[2]).asnumpy())

In [31]:
import mxnet as mx
num_epochs = 10
trainch3(net, train_iter, test_iter, loss, num_epochs, trainer)

In [33]:
num_epochs = 10
trainch3(net, train_iter, test_iter, loss, num_epochs, trainer)

In [36]:
num_epochs = 10
trainch3(net, train_iter, test_iter, loss, num_epochs, trainer)